In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
np.random.seed(15)
import json

In [2]:
data1= json.load(open('/content/drive/MyDrive/Datasets/Spotify-Million-Songs-Playlist-Collaborative-Filtering-Method/mpd.slice.0-999.json'))
data2= json.load(open('/content/drive/MyDrive/Datasets/Spotify-Million-Songs-Playlist-Collaborative-Filtering-Method/mpd.slice.1000-1999.json'))
data3= json.load(open('/content/drive/MyDrive/Datasets/Spotify-Million-Songs-Playlist-Collaborative-Filtering-Method/mpd.slice.2000-2999.json'))
data4= json.load(open('/content/drive/MyDrive/Datasets/Spotify-Million-Songs-Playlist-Collaborative-Filtering-Method/mpd.slice.3000-3999.json'))

In [3]:
df1= pd.DataFrame(data1['playlists'])
df2= pd.DataFrame(data2['playlists'])
df3= pd.DataFrame(data3['playlists'])
df4= pd.DataFrame(data4['playlists'])
df= pd.concat([df1, df2, df3, df4])

df= df.explode('tracks')
df= pd.concat([df.drop(['tracks'], axis=1), df['tracks'].apply(pd.Series)],axis=1) 

# renaming the columns with the same name
df.columns=['name', 'collaborative', 'pid', 'modified_at', 'num_tracks',
       'num_albums', 'num_followers', 'num_edits', 'playlist_duration_ms',
       'num_artists', 'description', 'pos', 'artist_name', 'track_uri',
       'artist_uri', 'track_name', 'album_uri', 'track_duration_ms',
       'album_name']

In [4]:
!pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import time

In [6]:
auth_manager = SpotifyClientCredentials(client_id='85d3f18d997e40629f507c5456edd141',
                                        client_secret='4fb6ed22c6724fd487911b94e42ab749')

In [7]:
#extract uri 
import re
df['track_uri']= df["track_uri"].apply(lambda x: re.findall(r'\w+$', str(x))[0])

In [8]:
sp = spotipy.Spotify(auth_manager=auth_manager)

In [9]:
def get_track_features(track_id):
  audio = sp.audio_features(track_id)
  track = sp.track(track_id)

  # Album Features
  release_date = track['album']['release_date']
  popularity = track['popularity']

  # Audio Features
  danceability = audio[0]['danceability']
  energy = audio[0]['energy']
  key = audio[0]['key']
  loudness = audio[0]['loudness']
  speechiness = audio[0]['speechiness']
  acousticness = audio[0]['acousticness']
  instrumentalness = audio[0]['instrumentalness']
  liveness = audio[0]['liveness']
  valence = audio[0]['valence']
  tempo = audio[0]['tempo']
  time_signature = audio[0]['time_signature']

  track_data = [release_date , popularity , danceability , energy , key , loudness , speechiness , acousticness , instrumentalness , liveness , valence , tempo , time_signature]
  return track_data

In [10]:
temp_df = df.iloc[255651:,:]  # updating index manually to extract features in batches
temp_df = temp_df.reset_index()

In [11]:
# counter variable 
i=0

# variable to print the progress
j=0

# list to append the features recieved from sporify API
track_features=[]

# for each track_uri in dataset
for uri in temp_df['track_uri']:

  if i<900:   # if i < 900 (you can choose any counter number) this if condition will send 900 request for features to spotify one by one for each track uri
  
    if uri == '656TZlNdVe90zHvmebFt9U' or uri == '5GiU7GOYjDH2yp7fMf9w9j' or uri == '2ZF3gpgGPjOfi1yTWGX6Bm' or uri == '2odGdanrUsgu0d2i2Q5XoT' or uri == '6tf7ENqpF89Xox3xyNPGMv' or uri == '0fJ6UuZ7onqcldGr8HT5QV' or uri == '0Jb7g80rInlj3fa6W65Bp2':
      # this confition is to avoid getting error 404 since these uri's are not available anymore on spotify API
      track_features.append([np.nan for i in range(13)]) # for these uri's we are appending NaN values and will deal with these sample later

    else: # if other uri's (which we can extract features from spotify API)
      track_features.append(get_track_features(uri)) # storing recieved features from spotify API to later use them to concatenate in out main data frame
      j+=1 # increment j for every succesfully recieved features to keep the track of how many samples are done to later print it

  else: # if i is greater than 900, it means that we now have made enough request consequtively to API, so we will sleep for some time and reset the counter to 0
    time.sleep(0.1)
    print(j)
    i=0

  i+=1 # incrementing i for every uri in dataset

900
1799
2698
3597
4496
5395
6294
7193
8092
8991
9890


In [12]:
len(track_features)

10724

In [13]:
track_features_df = pd.DataFrame(track_features, columns=['release_date','popularity','danceability','energy','key','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo','time_signature'])

In [15]:
temp_df_with_features = pd.concat([temp_df.iloc[:len(track_features_df),:],track_features_df],axis=1)

In [16]:
temp_df_with_features.shape

(10724, 33)

In [17]:
old_size = 255651

In [18]:
temp_df_with_features.to_csv("df("+str(old_size)+"+"+str(len(temp_df_with_features))+")"+".csv")

In [19]:
df1 = pd.read_csv('/content/drive/MyDrive/Datasets/df(0-'+str(old_size)+').csv')
df2 = pd.read_csv("df("+str(old_size)+"+"+str(len(temp_df_with_features))+")"+".csv")

df3 = pd.concat([df1,df2])

In [20]:
df3.drop(columns='Unnamed: 0',inplace=True)

In [21]:
print(df3.shape)
new_size = old_size + len(temp_df_with_features)

(266375, 33)


In [23]:
df3.to_csv('df(0-'+str(new_size)+').csv')

In [24]:
import shutil
shutil.copy('df(0-'+str(new_size)+').csv',"/content/drive/MyDrive/Datasets")


'/content/drive/MyDrive/Datasets/df(0-266375).csv'